dataset link: https://www.kaggle.com/puneet6060/intel-image-classification

In [51]:
import tensorflow as tf 
import pathlib
import numpy as np
import glob
import os
import keras
import random
from keras.applications.inception_resnet_v2 import InceptionResNetV2
import matplotlib.pyplot as plt


In [2]:
train_directory = "/content/drive/MyDrive/intel/data/seg_train/seg_train"
test_path = "/content/drive/MyDrive/intel/data/seg_test/seg_test"
test_path_2 = "/content/drive/MyDrive/intel/data/seg_pred/seg_pred"

In [3]:
train_dir = pathlib.Path(train_directory)
filenames = list(train_dir.glob('*/*.jpg'))
print("image count: " , len(filenames))
print("first image:" , str(filenames[0]) )

image count:  14034
first image: /content/drive/MyDrive/intel/data/seg_train/seg_train/buildings/19513.jpg


In [4]:
fnames = []
for f in filenames:
  fnames.append(str(f))
fnames[:5]

['/content/drive/MyDrive/intel/data/seg_train/seg_train/buildings/19513.jpg',
 '/content/drive/MyDrive/intel/data/seg_train/seg_train/buildings/19537.jpg',
 '/content/drive/MyDrive/intel/data/seg_train/seg_train/buildings/19466.jpg',
 '/content/drive/MyDrive/intel/data/seg_train/seg_train/buildings/19608.jpg',
 '/content/drive/MyDrive/intel/data/seg_train/seg_train/buildings/19526.jpg']

In [5]:
type(fnames)

list

In [6]:
random.shuffle(fnames)

In [7]:
test_dir = pathlib.Path(test_path)
test_filenames = list(test_dir.glob('*/*.jpg'))
test_fnames = []
for f in test_filenames : 
  test_fnames.append(str(f))
random.shuffle(test_fnames)

In [8]:
train_filelist = tf.data.Dataset.from_tensor_slices(fnames)

In [9]:
test_filelist = tf.data.Dataset.from_tensor_slices(test_fnames)

In [10]:
validation_size = 0.2

dataset_size = len(list(train_filelist))
print("Dataset size: ", dataset_size)
train_size = int((1-validation_size)*dataset_size)
print("train size: ", train_size)
train_data = train_filelist.take(train_size)
validation_data = train_filelist.skip(train_size)

Dataset size:  14034
train size:  11227


In [11]:
class_names = ["buildings", "forest", "glacier", "mountain", "sea", "street"]

In [12]:
def get_label(path):
  parts = tf.strings.split(path, os.path.sep)
  file_name = parts[-2]
  return file_name == class_names

In [13]:
def load_img(path):
  img = tf.io.read_file(path)
  img = tf.image.decode_jpeg(img,3)
    
  img = tf.image.convert_image_dtype(img, tf.float32)
  img = tf.image.resize(img, [150,150])
  return img

In [14]:
def Image_label(path):
  label = get_label(path)
  image = load_img(path)
  return image,label

In [15]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [16]:
train_dataset = train_filelist.map(Image_label , num_parallel_calls = AUTOTUNE)
#train_dataset = train_dataset.shuffle(1000)
train_dataset = train_dataset.batch(32)
train_dataset = train_dataset.prefetch(buffer_size = AUTOTUNE)

In [17]:
val_dataset = validation_data.map(Image_label , num_parallel_calls = AUTOTUNE)
val_dataset = val_dataset.batch(32)
val_dataset = val_dataset.prefetch(buffer_size = AUTOTUNE)

In [18]:
conv_base =tf.keras.applications.xception.Xception(
    include_top = False,
    input_shape = (150,150,3),
    weights="imagenet"
)

#conv_base.trainable = True

#set_trainable = False
#for layer in conv_base.layers:
#   if layer.name in ["block5_conv4" , "block5_conv3"]:
 #       set_trainable = True
  #  if set_trainable:
   #     layer.trainable = True
    #else:
     #   layer.trainable = False

for layer in conv_base.layers:
  layer.trainable = False

83697664/83683744 [==============================] - 0s 0us/step


In [19]:
model=tf.keras.models.Sequential()
model.add(conv_base)
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dropout(.4))
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dropout(.5))
model.add(keras.layers.Dense(6 , activation='softmax'))

In [21]:
model.compile(optimizer="adam",
              loss='CategoricalCrossentropy',
              metrics=tf.keras.metrics.AUC())
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 5, 5, 2048)        20861480  
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 2, 2, 2048)        0         
_________________________________________________________________
batch_normalization_4 (Batch (None, 2, 2, 2048)        8192      
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2097408   
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               6

In [22]:
conv_base.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 74, 74, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 74, 74, 32)   0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [23]:

history = model.fit(train_dataset,
                    steps_per_epoch= len(train_dataset)//32,
                    validation_data = val_dataset,
                    validation_steps = len(val_dataset)//32,
                    epochs=15)

Epoch 1/15
13/13 [==============================] - 103s 5s/step - loss: 1.6427 - auc: 0.8566 - val_loss: 0.5777 - val_auc: 0.9566
Epoch 2/15
13/13 [==============================] - 62s 5s/step - loss: 1.3978 - auc: 0.9192 - val_loss: 0.7453 - val_auc: 0.9543
Epoch 3/15
13/13 [==============================] - 73s 6s/step - loss: 1.6678 - auc: 0.9224 - val_loss: 1.3356 - val_auc: 0.9235
Epoch 4/15
13/13 [==============================] - 71s 5s/step - loss: 1.1586 - auc: 0.9269 - val_loss: 1.0366 - val_auc: 0.9418
Epoch 5/15
13/13 [==============================] - 71s 6s/step - loss: 1.3616 - auc: 0.9314 - val_loss: 0.5975 - val_auc: 0.9540
Epoch 6/15
13/13 [==============================] - 72s 6s/step - loss: 1.0304 - auc: 0.9352 - val_loss: 0.9720 - val_auc: 0.9274
Epoch 7/15
13/13 [==============================] - 72s 6s/step - loss: 0.9191 - auc: 0.9520 - val_loss: 0.4890 - val_auc: 0.9753
Epoch 8/15
13/13 [==============================] - 73s 6s/step - loss: 0.9463 - auc: 0.9

In [57]:
len(auc)

15

In [47]:
test_dataset = test_dataset.take(2000)
test_dataset = test_filelist.map(Image_label , num_parallel_calls = AUTOTUNE)

In [48]:
test_dataset = test_dataset.batch(2000)
#train_dataset.shape()

In [49]:
model.evaluate(test_dataset)

2/2 [==============================] - 34s 14s/step - loss: 0.4076 - auc: 0.9814


[0.4075828790664673, 0.9813719987869263]